In [14]:
import pandas as pd
import numpy as np
import random
import time
import datetime
import copy
import json
import math


In [15]:
#All Designed/Sponsored have devices
#Some aliens have devices institute that for alien powers

In [16]:
Character = {}
Character['Statistics'] = {}
Character['Origin'] = {}
Character['Powers'] = {}

In [17]:
def roll_statistic():
    statistic = np.random.randint(1,20)
    return statistic

def roll_luck():
    luck = 0
    roll = np.random.randint(1,100)
    if roll < 11:
        luck = 11 - roll
    return luck

def roll_origin():
    origin_dict = {}
    roll = np.random.randint(1,10)
    origin_dict['Artifact'] = "No"
    origin_dict['Lifespan'] = "Human"
    if roll <= 3:
        origin = "Mutant"
        age = np.random.randint(1,12) + 15
    elif roll == 4:
        origin = "Self Developed"
        age = np.random.randint(1,12) + 25
    elif roll == 5:
        origin = "Supernatural"
        age = np.random.randint(1,10) + 20
    elif roll == 6:
        origin = "Designed or Sponsored"
        age = np.random.randint(1,12) + 25
    elif roll == 10:
        origin = "Alien"
        age = np.random.randint(1,10) * np.random.randint(1,6)
        lifespan = np.random.randint(1,20) * np.random.randint(1,20)
    else:
        origin = "Accidental/Scientific"
        age = np.random.randint(1,8) * np.random.randint(1,6) + 25
        artifact_roll = np.random.randint(1,100)
        if artifact_roll <= 5:
            origin_dict['Artifact'] = "Yes"
        
    origin_dict['Origin'] = origin
    origin_dict['Age'] = age
    return origin_dict

def roll_effects(number, dice_sides):
    roll_total = 0
    for roll in range(number):
        dice_roll = np.random.randint(1,dice_sides)
        roll_total = roll_total + dice_roll
        
    return roll_total

In [18]:
Statistics = {"Strength":10,"Agility":10,"Charisma":10,"Intelligence":10,"Stamina":10,
              "PublicStanding":11,"Ego":11,"Luck":0}
Statistics_Abv = ['ST','AG','CH','IQ','SA','PS','EG','LK']

## Roll Main Statistics

Can reroll if main stats are not greater than 60

In [19]:
stats_total = 0
while stats_total <= 60:
    stats_total = 0
    for index, key in enumerate(Statistics.keys()):
        if index < 5:
            Statistics[key] = roll_statistic()
            stats_total = stats_total + Statistics[key]
            print(key, Statistics[key])
    print("Total:", stats_total)

Strength 18
Agility 4
Charisma 8
Intelligence 19
Stamina 15
Total: 64


## Roll Other Statistics

In [20]:
Statistics['Ego'] = roll_statistic()
Statistics['Luck'] = roll_luck()

Statistics

{'Strength': 18,
 'Agility': 4,
 'Charisma': 8,
 'Intelligence': 19,
 'Stamina': 15,
 'PublicStanding': 11,
 'Ego': 18,
 'Luck': 0}

## Roll Origin

In [21]:
Origin = roll_origin()
Origin

{'Artifact': 'No',
 'Lifespan': 'Human',
 'Origin': 'Accidental/Scientific',
 'Age': 37}

In [22]:
df = pd.read_csv('data/powers.csv')

## Roll Powers

### Number of Powers

In [23]:
Powers_roll = df['Powers'].sample()
Powers = {}
Powers['Number'] = Powers_roll.values[0]

if Statistics['Luck'] > 0:
    if Statistics['Luck'] == 10:
        Powers = Powers + 1
        roll = np.random.randint(1,100)
        if roll <= 25:
            Powers = Powers + 1
    elif Statistics['Luck'] <= 9 and Statistics['Luck'] >= 7:
        roll = np.random.randint(1,100)
        if roll <= 75:
            Powers = Powers + 1
    elif Statistics['Luck'] <= 6 and Statistics['Luck'] >= 4:
        roll = np.random.randint(1,100)
        if roll <= 50:
            Powers = Powers + 1
    else:
        roll = np.random.randint(1,100)
        if roll <= 25:
            Powers = Powers + 1
    
Powers    

{'Number': 3}

In [24]:
Powers['List'] = []
for power in range(Powers['Number']):
    Power_roll = df[Origin['Origin']].sample().values[0]
    print(Power_roll)
    Powers['List'].append(Power_roll)
        

Weather Control
Enhanced Stamina
Dimensional Gate


In [45]:
Powers['Detail'] = {}
for power in Powers['List']:
    Powers['Detail'][power] = {}

In [27]:
Statistics

{'Strength': 18,
 'Agility': 4,
 'Charisma': 8,
 'Intelligence': 19,
 'Stamina': 15,
 'PublicStanding': 11,
 'Ego': 18,
 'Luck': 0}

In [28]:
Powers

{'Number': 3,
 'List': ['Weather Control', 'Enhanced Stamina', 'Dimensional Gate']}

In [29]:
Character['Statistics'] = Statistics
Character['Origin'] = Origin
Character['Powers'] = Powers

for key in Character['Statistics'].keys():
    Character[key + '_Effects'] = {}
Character

{'Statistics': {'Strength': 18,
  'Agility': 4,
  'Charisma': 8,
  'Intelligence': 19,
  'Stamina': 15,
  'PublicStanding': 11,
  'Ego': 18,
  'Luck': 0},
 'Origin': {'Artifact': 'No',
  'Lifespan': 'Human',
  'Origin': 'Accidental/Scientific',
  'Age': 37},
 'Powers': {'Number': 3,
  'List': ['Weather Control', 'Enhanced Stamina', 'Dimensional Gate']},
 'Strength_Effects': {},
 'Agility_Effects': {},
 'Charisma_Effects': {},
 'Intelligence_Effects': {},
 'Stamina_Effects': {},
 'PublicStanding_Effects': {},
 'Ego_Effects': {},
 'Luck_Effects': {}}

In [31]:
dfc = pd.read_csv('data/characteristics.csv', low_memory=False)

## Characteristic Effects
### note that this doesn't take any powers into account currently

In [32]:
stat = 'Strength'
Character[stat + '_Effects']['Description'] = dfc[stat].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['HT'] = dfc[stat + '_HT'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['DD'] = dfc[stat + '_DD'].iloc[Character['Statistics'][stat]]

if 'd' in Character[stat + '_Effects']['HT']:
    if '+' not in Character[stat + '_Effects']['HT']:
        roll_list = Character[stat + '_Effects']['HT'].split('d')
        Character[stat + '_Effects']['HT'] = roll_effects( int(roll_list[0]), int(roll_list[1]))
    else:
        roll_list_plus = Character[stat + '_Effects']['HT'].split('+')
        roll_list = roll_list_plus[0].split('d')
        Character[stat + '_Effects']['HT'] = roll_effects( int(roll_list[0]), int(roll_list[1])) + int(roll_list_plus[1])
if 'd' in Character[stat + '_Effects']['DD']:
    if '+' not in Character[stat + '_Effects']['DD']:
        roll_list = Character[stat + '_Effects']['DD'].split('d')
        Character[stat + '_Effects']['DD'] = roll_effects( int(roll_list[0]), int(roll_list[1]))
    else:
        roll_list_plus = Character[stat + '_Effects']['DD'].split('+')
        roll_list = roll_list_plus[0].split('d')
        Character[stat + '_Effects']['DD'] = roll_effects( int(roll_list[0]), int(roll_list[1])) + int(roll_list_plus[1])

stat = 'Agility'
Character[stat + '_Effects']['Description'] = dfc[stat].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['Move'] = dfc[stat + '_Move'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['Accuracy'] = dfc[stat + '_Accuracy'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['HT'] = dfc[stat + '_HT'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['DD'] = dfc[stat + '_DD'].iloc[Character['Statistics'][stat]]

stat = 'Charisma'
Character[stat + '_Effects']['ReactionHero'] = dfc[stat + '_ReactionHero'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['ReactionVillain'] = dfc[stat + '_ReactionVillain'].iloc[Character['Statistics'][stat]]

stat = 'Intelligence'
Character[stat + '_Effects']['Accuracy'] = dfc[stat + '_Accuracy'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['HT'] = dfc[stat + '_HT'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['DD'] = dfc[stat + '_DD'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['DetectEntrances'] = dfc[stat + '_DetectEntrances'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['DetectTraps'] = dfc[stat + '_DetectTraps'].iloc[Character['Statistics'][stat]]

if 'd' in Character[stat + '_Effects']['DetectEntrances']:
    roll_list = Character[stat + '_Effects']['DetectEntrances'].split('d')
    Character[stat + '_Effects']['DetectEntrances'] = roll_effects( int(roll_list[0]), int(roll_list[1]))

if 'd' in Character[stat + '_Effects']['DetectTraps']:
    roll_list = Character[stat + '_Effects']['DetectTraps'].split('d')
    Character[stat + '_Effects']['DetectTraps'] = roll_effects( int(roll_list[0]), int(roll_list[1]))
    

stat = 'Ego'
Character[stat + '_Effects']['Description'] = dfc[stat].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['CompulsoryRetreat'] = dfc[stat + '_CompulsoryRetreat'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['WillingRetreat'] = dfc[stat + '_WillingRetreat'].iloc[Character['Statistics'][stat]]
Character[stat + '_Effects']['HP'] = dfc[stat + '_HP'].iloc[Character['Statistics'][stat]]

stat = 'PublicStanding'
Character[stat + '_Effects']['ReactionDM'] = dfc[stat + '_ReactionDM'].iloc[Character['Statistics'][stat]]

    
Character

{'Statistics': {'Strength': 18,
  'Agility': 4,
  'Charisma': 8,
  'Intelligence': 19,
  'Stamina': 15,
  'PublicStanding': 11,
  'Ego': 18,
  'Luck': 0},
 'Origin': {'Artifact': 'No',
  'Lifespan': 'Human',
  'Origin': 'Accidental/Scientific',
  'Age': 37},
 'Powers': {'Number': 3,
  'List': ['Weather Control', 'Enhanced Stamina', 'Dimensional Gate']},
 'Strength_Effects': {'Description': 'Strength of two men',
  'HT': '1',
  'DD': 3},
 'Agility_Effects': {'Description': 'Slow',
  'Move': 45,
  'Accuracy': -10,
  'HT': -1,
  'DD': 0},
 'Charisma_Effects': {'ReactionHero': 5, 'ReactionVillain': -5},
 'Intelligence_Effects': {'Accuracy': 10,
  'HT': 2,
  'DD': 1,
  'DetectEntrances': 20,
  'DetectTraps': 27},
 'Stamina_Effects': {},
 'PublicStanding_Effects': {'ReactionDM': 0},
 'Ego_Effects': {'Description': 'Conceited',
  'CompulsoryRetreat': 0,
  'WillingRetreat': 20,
  'HP': 15},
 'Luck_Effects': {}}

In [33]:
dfc.dtypes

Characteristic                    int64
Strength                         object
Strength_HT                      object
Strength_DD                      object
Agility                          object
Agility_Move                      int64
Agility_Accuracy                  int64
Agility_HT                        int64
Agility_DD                        int64
Charisma                         object
Charisma_ReactionHero             int64
Charisma_ReactionVillain          int64
Intelligence                     object
Intelligence_HT                   int64
Intelligence_DD                   int64
Intelligence_Accuracy             int64
Intelligence_DetectEntrances     object
Intelligence_DetectTraps         object
Ego                              object
Ego_CompulsoryRetreat             int64
Ego_WillingRetreat                int64
Ego_HP                            int64
PublicStanding_ReactionDM         int64
Height                          float64
Height_WeightDM                  object


## Calculated Statistics

In [34]:
def normal_round(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

HitPoints = int(normal_round(float(Character['Statistics']['Stamina'])/2))
HitPoints = HitPoints + np.random.randint(1,10)
HitPoints = HitPoints + Character['Statistics']['Luck']
HitPoints = HitPoints + int(Character['Strength_Effects']['HT'])
HitPoints = HitPoints + int(Character['Agility_Effects']['HT'])
HitPoints = HitPoints + int(Character['Intelligence_Effects']['HT'])

Character['HitPoints'] = HitPoints
Character['HitPoints']

14

In [35]:
ActionPotential = Character['Statistics']['Strength']
ActionPotential = ActionPotential + int(normal_round(float(Character['Statistics']['Intelligence'])/2))
ActionPotential = ActionPotential + Character['Statistics']['Stamina']
ActionPotential = ActionPotential + int(normal_round(float(Character['Statistics']['Agility'])/2))
ActionPotential = ActionPotential + int(normal_round(float(Character['Statistics']['Ego'])/2))
ActionPotential = ActionPotential + + Character['Statistics']['Luck']

Character['ActionPotential'] = ActionPotential
Character['ActionPotential']

54

In [36]:
DirectDamage = int(Character['Strength_Effects']['DD'])
DirectDamage = DirectDamage + int(Character['Agility_Effects']['DD'])
DirectDamage = DirectDamage + int(Character['Intelligence_Effects']['DD'])

Character['DirectDamage'] = DirectDamage
Character['DirectDamage']

4

In [37]:
sex_roll = np.random.randint(1,100)
if sex_roll <= 50:
    Character['Sex'] = 'Female'
    height_mod = roll_effects(5,4) * -1
else:
    Character['Sex'] = 'Male'
    height_mod = 0
    
Height_roll = dfc['Height'].sample()
Height_roll

39    182.5
Name: Height, dtype: float64

In [38]:
WeightDM = dfc.iloc[Height_roll.index]['Height_WeightDM'].values[0]
if 'd' in WeightDM:
    roll_list = WeightDM.split('d')
    weight_mod = roll_effects( int(roll_list[0]), int(roll_list[1]))
    if Character['Height'] < 190:
        weight_mod = weight_mod * -1
else:
    weight_mod = int(WeightDM)
    
Weight_roll = dfc['Weight'].sample().values[0]

Character['Height'] = Height_roll.values[0] + height_mod

Character['Weight'] = int(Weight_roll) + weight_mod
Character['Weight']

60

## Profession

In [39]:
Job_roll = dfc['Job'].sample()

Character['Job'] = Job_roll.values[0]

Pay = dfc.iloc[Job_roll.index]['Pay'].values[0]
if 'd' in Pay:
    roll_list_plus = Pay.split('+')
    roll_list = roll_list_plus[1].split('d')
    Character['Pay'] = roll_effects( int(roll_list[0]), int(roll_list[1])) + int(roll_list_plus[0])
    
PatrolDM = dfc.iloc[Job_roll.index]['PatrolDM'].values[0]    

if 'Other' in Character['Job']:
    other_roll = np.random.randint(1,6)
    if other_roll == 1:
        Character['Job'] = 'Supergroup'
    elif other_roll == 2:
        Character['Job'] = 'Mercenary'
    elif other_roll == 3:
        Character['Job'] = 'Spy'
    elif other_roll == 4:
        Character['Job'] = 'Millionaire'
    elif other_roll == 5:
        Character['Job'] = 'Alien Scout or God'
    else:
        Character['Job'] = 'Supernatural Investigator'

if 'NPC' in Character['Job']:
    roll_high = np.random.randint(1,6)
    if roll_high <= 3:
        Job_npc_roll = dfc['JobNPCLow'].sample()
    else:
        Job_npc_roll = dfc['JobNPCHight'].sample()
    Character['Job'] = Job_npc_roll.values[0]

Characters can start with a skill appropriate to their profession from the table

### Other Skill

In [40]:
Skill_roll = np.random.randint(1,100)
if Skill_roll < ( int(Character['Origin']['Age']) + int(Character['Statistics']['Luck']) ):
    Character['OtherSkill'] = "Yes"
else:
    Character['OtherSkill'] = "No"

In [46]:
Character

{'Statistics': {'Strength': 18,
  'Agility': 4,
  'Charisma': 8,
  'Intelligence': 19,
  'Stamina': 15,
  'PublicStanding': 11,
  'Ego': 18,
  'Luck': 0},
 'Origin': {'Artifact': 'No',
  'Lifespan': 'Human',
  'Origin': 'Accidental/Scientific',
  'Age': 37},
 'Powers': {'Number': 3,
  'List': ['Weather Control', 'Enhanced Stamina', 'Dimensional Gate'],
  'Detail': {'Weather Control': {},
   'Enhanced Stamina': {},
   'Dimensional Gate': {}}},
 'Strength_Effects': {'Description': 'Strength of two men',
  'HT': '1',
  'DD': 3},
 'Agility_Effects': {'Description': 'Slow',
  'Move': 45,
  'Accuracy': -10,
  'HT': -1,
  'DD': 0},
 'Charisma_Effects': {'ReactionHero': 5, 'ReactionVillain': -5},
 'Intelligence_Effects': {'Accuracy': 10,
  'HT': 2,
  'DD': 1,
  'DetectEntrances': 20,
  'DetectTraps': 27},
 'Stamina_Effects': {},
 'PublicStanding_Effects': {'ReactionDM': 0},
 'Ego_Effects': {'Description': 'Conceited',
  'CompulsoryRetreat': 0,
  'WillingRetreat': 20,
  'HP': 15},
 'Luck_Effect

In [9]:
from super_squadron import powers as pw
pw.printtest

<function super_squadron.powers.printtest()>

In [47]:
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

with open('character_test.json', 'w') as f:
    json.dump(Character, f, default=np_encoder)
